In [ ]:
# https://www.kaggle.com/datasets/noodulz/pokemon-dataset-1000/code

In [66]:
import os
import cv2

import pandas as pd
import numpy as np
import tensorflow as tf

from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [67]:
def pre_process_and_build_image_dataset(folder_path):
    rotation = {
        "90_clockwise": cv2.ROTATE_90_CLOCKWISE,
        "180": cv2.ROTATE_180,
        "90_counter_clockwise": cv2.ROTATE_90_COUNTERCLOCKWISE
    }
    
    for root, dirs, files in os.walk(folder_path):
        for poke in dirs:
            for image in os.listdir(f"{root}/{poke}"):
                original_image_path = f"{root}/{poke}/{image}"
                
                img = cv2.imread(original_image_path, cv2.IMREAD_GRAYSCALE)
                img = cv2.resize(img, (32, 32))
                
                # save original image in grayscale
                cv2.imwrite(original_image_path, img)

In [68]:
train_path = "./data/train"
test_path = "./data/test"
val_path = "./data/val"

In [69]:
pre_process_and_build_image_dataset(train_path)
pre_process_and_build_image_dataset(test_path)
pre_process_and_build_image_dataset(val_path)

In [70]:
train_dataset = image_dataset_from_directory(
    train_path,
    label_mode="categorical",
    image_size=(32, 32),
    seed=42
)

Found 20921 files belonging to 1000 classes.


In [71]:
test_dataset = image_dataset_from_directory(
    test_path,
    label_mode="categorical",
    image_size=(32, 32),
    seed=42
)

Found 3239 files belonging to 1000 classes.


In [72]:
val_dataset = image_dataset_from_directory(
    val_path,
    label_mode="categorical",
    image_size=(32, 32),
    seed=42
)

Found 2379 files belonging to 898 classes.


In [79]:
model = models.Sequential([
    layers.Input(shape=(32, 32, 3)),
    
    layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
    layers.MaxPool2D(pool_size=(2, 2)),
    
    layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPool2D(pool_size=(2, 2)),
    
    layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
    
    layers.Conv2D(filters=256, kernel_size=(3, 3), activation='relu'),
    
    layers.Flatten(),
    layers.Dense(256, activation="relu"),
    layers.Dense(512, activation="relu"),
    layers.Dense(1024, activation="relu"),
    layers.Dense(2048, activation="relu"),
    layers.Dense(1000, activation="softmax")
])

In [80]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

In [81]:
model.fit(train_dataset, epochs=50, batch_size=64)

Epoch 1/50
654/654 ━━━━━━━━━━━━━━━━━━━━ 33s 47ms/step - accuracy: 0.0010 - loss: 0.0256
Epoch 2/50
654/654 ━━━━━━━━━━━━━━━━━━━━ 26s 39ms/step - accuracy: 0.0021 - loss: 0.0079
Epoch 3/50
654/654 ━━━━━━━━━━━━━━━━━━━━ 25s 38ms/step - accuracy: 0.0100 - loss: 0.0073
Epoch 4/50
654/654 ━━━━━━━━━━━━━━━━━━━━ 25s 38ms/step - accuracy: 0.0477 - loss: 0.0064
Epoch 5/50
654/654 ━━━━━━━━━━━━━━━━━━━━ 25s 38ms/step - accuracy: 0.1281 - loss: 0.0054
Epoch 6/50
654/654 ━━━━━━━━━━━━━━━━━━━━ 25s 38ms/step - accuracy: 0.2395 - loss: 0.0045
Epoch 7/50
654/654 ━━━━━━━━━━━━━━━━━━━━ 25s 38ms/step - accuracy: 0.3508 - loss: 0.0038
Epoch 8/50
654/654 ━━━━━━━━━━━━━━━━━━━━ 25s 37ms/step - accuracy: 0.4692 - loss: 0.0031
Epoch 9/50
654/654 ━━━━━━━━━━━━━━━━━━━━ 25s 38ms/step - accuracy: 0.5708 - loss: 0.0026
Epoch 10/50
654/654 ━━━━━━━━━━━━━━━━━━━━ 25s 38ms/step - accuracy: 0.6424 - loss: 0.0022
Epoch 11/50
654/654 ━━━━━━━━━━━━━━━━━━━━ 25s 38ms/step - accuracy: 0.6971 - loss: 0.0019
Epoch 12/50
654/654 ━━━━━━━━━━

In [82]:
model.evaluate(test_dataset)

102/102 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.7591 - loss: 0.0032


[0.003443257650360465, 0.743130624294281]